In [1]:
import pandas as pd
from soccerdata import Understat       
import csv

[07/03/25 16:06:45] INFO     No custom team name replacements found. You can configure these in       ]8;id=260699;file://c:\project\module_project\.venv\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=872957;file://c:\project\module_project\.venv\lib\site-packages\soccerdata\_config.py#91\91]8;;\
                             C:\Users\User\soccerdata\config\teamname_replacements.json.                           

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=302118;file://c:\project\module_project\.venv\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=747202;file://c:\project\module_project\.venv\lib\site-packages\soccerdata\_config.py#197\197]8;;\
                             C:\Users\User\soccerdata\config\league_dict.json.                                     

In [2]:
# ────────────────────────────────────────
# 1) Understat 2016~2025 EPL 경기 로드
# ────────────────────────────────────────
LEAGUE  = ["ENG-Premier League"]
SEASONS = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]                    # 2016 ~ 2025

us = Understat(leagues=LEAGUE, seasons=SEASONS)
stats = us.read_team_match_stats()          

# # 필요한 컬럼만
stats = stats[['game_id', 'date','home_team', 'away_team', 'home_goals', 'away_goals', 'home_expected_points', 'away_expected_points', 'home_xg', 'away_xg', 'home_ppda', 'away_ppda', 'home_deep_completions', 'away_deep_completions']]

stats.info()

                    INFO     Saving cached data to C:\Users\User\soccerdata\data\Understat           ]8;id=969283;file://c:\project\module_project\.venv\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=246219;file://c:\project\module_project\.venv\lib\site-packages\soccerdata\_common.py#263\263]8;;\

                    WARNING  c:\project\module_project\.venv\lib\site-packages\soccerdata\_common.p ]8;id=857332;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=358591;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             y:145: UserWarning: Season id "2021" is ambiguous: interpreting as                    
                             "20-21"                                                                               
                               warnings.warn(msg, stacklevel=1)                                                    
                                                                                                                   

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3040 entries, ('ENG-Premier League', '1617', '2016-08-13 Burnley-Swansea') to ('ENG-Premier League', '2425', '2025-05-25 Wolverhampton Wanderers-Brentford')
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   game_id                3040 non-null   Int64         
 1   date                   3040 non-null   datetime64[ns]
 2   home_team              3040 non-null   string        
 3   away_team              3040 non-null   string        
 4   home_goals             3040 non-null   Int64         
 5   away_goals             3040 non-null   Int64         
 6   home_expected_points   3040 non-null   Float64       
 7   away_expected_points   3040 non-null   Float64       
 8   home_xg                3040 non-null   Float64       
 9   away_xg                3040 non-null   Float64       
 10  home_ppda              3040 non-null   Float64 

In [ ]:
# 2-4 데이터 전처리 : Standardize(표준화), OneHotEncoding(원-핫 인코딩)
log_columns = ['HomeElo', 'AwayElo'] # 로그 변환할 columns
standarize_columns = ['HomeElo', 'AwayElo', 'GF3Home', 'GA3Home', 'GF5Home', 'GA5Home', 'GF3Away', 'GA3Away', 'GF5Away', 'GA5Away']  # 표준화할 columns
encoding_columns = ['HomeTeam', 'AwayTeam']  # 원-핫 인코딩할 columns

# 분산이 큰 배당률 관련 columns와 Elo columns는 표준화 전에 log scale을 먼저 적용
data_final[log_columns] = np.log1p(data_final[log_columns])

# Pipeline에서 전처리 해줄 ColumnsTransformer 정의
preprocessor = ColumnTransformer(
    transformers=[
        ('standardize', StandardScaler(), standarize_columns),  # 표준화
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True), encoding_columns)  # 원-핫 인코딩
    ],
    remainder='passthrough'  # 나머지 컬럼은 그대로 유지
)


In [3]:
# 1) 날짜 컬럼 → 연-월-일만 남기고 ‘MatchDate’로
stats['MatchDate'] = pd.to_datetime(stats['date']).dt.date

# 2) 팀 이름 컬럼 → 기존 데이터와 맞춰서 대문자 표기
stats = stats.rename(columns={
    'home_team': 'HomeTeam',
    'away_team': 'AwayTeam'
})

# 3) 불필요해진 원본 'date' 컬럼 제거
stats = stats.drop(columns=['date'])

In [4]:
# 0) 준비
base_cols = ['game_id', 'MatchDate', 'HomeTeam','AwayTeam']
metric_pairs = [
    ('goals',             'home_goals',            'away_goals'),
    ('expected_points',   'home_expected_points',  'away_expected_points'),
    ('xg',                'home_xg',               'away_xg'),
    ('ppda',              'home_ppda',             'away_ppda'),
    ('deep_completions',  'home_deep_completions', 'away_deep_completions')
]
windows = (3, 5)

# 1) long 포맷으로 변환  (team · side · metric1 …)
long_rows = []
for side_lbl, team_col in [('home','HomeTeam'), ('away','AwayTeam')]:
    tmp = stats[[team_col,'MatchDate', 'game_id']].rename(columns={team_col:'Team'})
    tmp['side'] = side_lbl
    # 붙일 메트릭들
    for name, h_col, a_col in metric_pairs:
        tmp[name] = stats[h_col] if side_lbl=='home' else stats[a_col]
    long_rows.append(tmp)

long_df = pd.concat(long_rows, ignore_index=True)
long_df = long_df.sort_values(['Team','MatchDate'])

In [5]:
# 팀별 평균 (shift(1) ⇒ 현재 경기 제외)
metrics = [m[0] for m in metric_pairs]

def add_roll(df):
    df = df.copy()
    for w in windows:
        roll = (
            df[metrics]
              .rolling(window=w, min_periods=1)
              .mean()
              .shift(1)
        )
        roll.columns = [f'{c}_l{w}' for c in roll.columns]
        df = pd.concat([df, roll], axis=1)
    return df

long_roll = (
    long_df
      .groupby('Team', group_keys=False)
      .apply(add_roll)
)

[07/03/25 16:06:46] WARNING  C:\Users\User\AppData\Local\Temp\ipykernel_16268\2656111271.py:18:     ]8;id=516918;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=925661;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             FutureWarning: DataFrameGroupBy.apply operated on the grouping                        
                             columns. This behavior is deprecated, and in a future version of                      
                             pandas the grouping columns will be excluded from the operation.                      
                             Either pass `include_groups=False` to exclude the groupings or                        
                             explicitly select the grouping columns after groupby to silence this                  
                             warning.                                                                              
                               long_df                                                                             
                                                                                                                   

In [6]:
# 3) 홈·원정 다시 wide 형식으로 pivot
roll_cols = [c for c in long_roll.columns if c.endswith(('l3','l5'))]

home_df = (
    long_roll[long_roll['side'] == 'home']
      [['game_id', 'MatchDate', 'Team', *roll_cols]]
      .rename(columns={**{c: f'home_{c}' for c in roll_cols},
                       'Team': 'HomeTeam'})
)

away_df = (
    long_roll[long_roll['side'] == 'away']
      [['game_id', 'MatchDate', 'Team', *roll_cols]]
      .rename(columns={**{c: f'away_{c}' for c in roll_cols},
                       'Team': 'AwayTeam'})
)

feat_df = home_df.merge(away_df, on=['game_id','MatchDate'], how='inner')


In [7]:
# 1) 홈팀 롤링 피처
home_cols = [c for c in feat_df.columns if c.startswith('home_')]
home_feat = (feat_df[['game_id','MatchDate','HomeTeam', *home_cols]]
             .drop_duplicates())

stats = stats.merge(
    home_feat,
    on=['game_id','MatchDate','HomeTeam'],
    how='left'
)

# 2) 원정팀 롤링 피처
away_cols = [c for c in feat_df.columns if c.startswith('away_')]
away_feat = (feat_df[['game_id','MatchDate','AwayTeam', *away_cols]]
             .drop_duplicates())

stats = stats.merge(
    away_feat,
    on=['game_id','MatchDate','AwayTeam'],
    how='left'
)

In [8]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3040 entries, 0 to 3039
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   game_id                   3040 non-null   Int64  
 1   HomeTeam                  3040 non-null   string 
 2   AwayTeam                  3040 non-null   string 
 3   home_goals                3040 non-null   Int64  
 4   away_goals                3040 non-null   Int64  
 5   home_expected_points      3040 non-null   Float64
 6   away_expected_points      3040 non-null   Float64
 7   home_xg                   3040 non-null   Float64
 8   away_xg                   3040 non-null   Float64
 9   home_ppda                 3040 non-null   Float64
 10  away_ppda                 3040 non-null   Float64
 11  home_deep_completions     3040 non-null   Int64  
 12  away_deep_completions     3040 non-null   Int64  
 13  MatchDate                 3040 non-null   object 
 14  home_goa

In [ ]:
stats_clean = stats[['MatchDate', 'HomeTeam', 'AwayTeam', 'home_xg_l3', 'home_xg_l5', 'away_xg_l3', 'away_xg_l5', 'home_ppda_l3', 'home_ppda_l5', 'away_ppda_l3', 'away_ppda_l5', 'home_deep_completions_l3', 'home_deep_completions_l5', 'away_deep_completions_l3', 'away_deep_completions_l5']].copy()

In [11]:
stats_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3040 entries, 0 to 3039
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   MatchDate                 3040 non-null   object 
 1   HomeTeam                  3040 non-null   string 
 2   AwayTeam                  3040 non-null   string 
 3   home_xg_l3                3025 non-null   float64
 4   home_xg_l5                3025 non-null   float64
 5   away_xg_l3                3021 non-null   float64
 6   away_xg_l5                3021 non-null   float64
 7   home_ppda_l3              3025 non-null   float64
 8   home_ppda_l5              3025 non-null   float64
 9   away_ppda_l3              3021 non-null   float64
 10  away_ppda_l5              3021 non-null   float64
 11  home_deep_completions_l3  3025 non-null   float64
 12  home_deep_completions_l5  3025 non-null   float64
 13  away_deep_completions_l3  3021 non-null   float64
 14  away_dee

In [12]:
stats_clean.to_csv('xg_data.csv', index=False)